In [1]:
import mysql.connector
import collections

class Book:
    def __init__(self, book_id, title, author, genre, price, quantity_in_stock):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.genre = genre
        self.price = price
        self.quantity_in_stock = quantity_in_stock

class Library:
    def __init__(self, db_config):
        self.db = mysql.connector.connect(**db_config)
        self.cursor = self.db.cursor(dictionary=True)

    def add_book(self, book):
        query = "INSERT INTO Books (book_id, title, author, genre, price, quantity_in_stock) VALUES (%s, %s, %s, %s, %s, %s)"
        values = (book.book_id, book.title, book.author, book.genre, book.price, book.quantity_in_stock)
        self.cursor.execute(query, values)
        self.db.commit()

    def update_book(self, book_id, title=None, author=None, genre=None, price=None, quantity_in_stock=None):
        current_book_query = "SELECT * FROM Books WHERE book_id=%s"
        self.cursor.execute(current_book_query, (book_id,))
        current_book = self.cursor.fetchone()
        
        if not current_book:
            print("Book ID not found.")
            return

        title = title if title else current_book['title']
        author = author if author else current_book['author']
        genre = genre if genre else current_book['genre']
        price = price if price else current_book['price']
        quantity_in_stock = quantity_in_stock if quantity_in_stock else current_book['quantity_in_stock']

        update_query = """
        UPDATE Books SET title=%s, author=%s, genre=%s, price=%s, quantity_in_stock=%s WHERE book_id=%s
        """
        self.cursor.execute(update_query, (title, author, genre, price, quantity_in_stock, book_id))
        self.db.commit()
        print("Book updated successfully in the database.")
        
    def delete_book(self, book_id):
        query = "DELETE FROM Books WHERE book_id=%s"
        self.cursor.execute(query, (book_id,))
        self.db.commit()

    def check_availability(self, book_id):
        query = "SELECT quantity_in_stock FROM Books WHERE book_id=%s"
        self.cursor.execute(query, (book_id,))
        result = self.cursor.fetchone()
        if result and result['quantity_in_stock'] > 0:
            return True
        return False

    def update_quantity(self, book_id, quantity_change):
        query = "UPDATE Books SET quantity_in_stock = quantity_in_stock + %s WHERE book_id=%s"
        self.cursor.execute(query, (quantity_change, book_id))
        self.db.commit()
        
    def all_stock(self):
        query = "Select * from Books"
        self.cursor.execute(query)
        return self.cursor.fetchall()
        
    def low_stock_report(self):
        query = "SELECT * FROM Books WHERE quantity_in_stock < 5"
        self.cursor.execute(query)
        return self.cursor.fetchall()

def main():
    correct_password = 'Beta2'
    correct_database = 'library'

    while True:
        password = input("Enter the database password: ")
        database = input("Enter the database name: ")

        if password == correct_password and database == correct_database:
            print("Access granted.")
            db_config = {
                'user': 'root',
                'password': password,
                'host': 'localhost',
                'database': database
            }
            
            break
        else:
            print("Error: Incorrect password or database name. Please try again.")

    
    library = Library(db_config)

    while True:
        print("Library Management System")
        print("1. Add a Book")
        print("2. Update a Book")
        print("3. Delete a Book")
        print("4. Check for Availability")
        print("5. Update Quantity")
        print("6. Low Stock Report")
        print("7. ALL Book details")
        print("8. Exit")
        choice = input("Please enter your choice: ")

        try:
            if choice == '1':
                book_id = int(input("Please enter book ID: "))
                title = input("Please enter title: ")
                author = input("Please enter author: ")
                genre = input("Please enter genre: ")
                price = float(input("Please enter price: "))
                quantity_in_stock = int(input("Please enter quantity in stock: "))
                book = Book(book_id, title, author, genre, price, quantity_in_stock)
                library.add_book(book)
                print("Book added successfully in the database.")

            elif choice == '2':
                book_id = int(input("Enter book ID: "))
                title = input("Please enter new title (leave blank to keep current): ") or None
                author = input("Please enter new author (leave blank to keep current): ") or None
                genre = input("Please enter new genre (leave blank to keep current): ") or None
                price = input("Please enter new price (leave blank to keep current): ")
                price = float(price) if price else None
                quantity_in_stock = input("Please enter new quantity in stock (leave blank to keep current): ")
                quantity_in_stock = int(quantity_in_stock) if quantity_in_stock else None
                library.update_book(book_id, title, author, genre, price, quantity_in_stock)
                print("Book updated successfully in the database.")

            elif choice == '3':
                book_id = int(input("Please enter book ID: "))
                library.delete_book(book_id)
                print("Book deleted successfully in the database.")

            elif choice == '4':
                book_id = int(input("Please enter book ID: "))
                available = library.check_availability(book_id)
                print("Book is available." if available else "Book is not available.")

            elif choice == '5':
                book_id = int(input("Please enter book ID: "))
                quantity_change = int(input("Enter quantity change (negative for borrowing, positive for returning): "))
                library.update_quantity(book_id, quantity_change)
                print("Quantity updated successfully in the database.")

            elif choice == '6':
                low_stock_books = library.low_stock_report()
                if not low_stock_books:
                    print("No books are low in stock.")
                else:
                    print("Books low in stock:")
                    for book in low_stock_books:
                        print(f"ID: {book['book_id']}, Title: {book['title']}, Author: {book['author']}, Genre: {book['genre']}, Price: {book['price']}, Quantity: {book['quantity_in_stock']}")

            elif choice == '7':
                all_stock_book = library.all_stock()
                if not all_stock_book:
                    print("No books in database.")
                else:
                    print("Books are in stock:")
                    for book in all_stock_book:
                        print(f"Name of the book:{book['title']}, Quatity: {book['quantity_in_stock']} \n")
                
            elif choice == '8':
                break

            else:
                print("Invalid choice. Please try again.")

        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    main()


Enter the database password: Beta2
Enter the database name: library
Access granted.
Library Management System
1. Add a Book
2. Update a Book
3. Delete a Book
4. Check for Availability
5. Update Quantity
6. Low Stock Report
7. ALL Book details
8. Exit
Please enter your choice: 7
Books are in stock:
Name of the book:Dawn of heroes, Quatity: 250 

Library Management System
1. Add a Book
2. Update a Book
3. Delete a Book
4. Check for Availability
5. Update Quantity
6. Low Stock Report
7. ALL Book details
8. Exit
Please enter your choice: 6
No books are low in stock.
Library Management System
1. Add a Book
2. Update a Book
3. Delete a Book
4. Check for Availability
5. Update Quantity
6. Low Stock Report
7. ALL Book details
8. Exit
Please enter your choice: 8
